# 문제 1

## 지도 학습과 비지도 학습의 알고리즘과 특징에 대해 서술하시오.

- 지도학습이란 feature data 와 실제 결과값 모두 입력값으로 사용하여, 알고리즘을 training 시킴으로써 최적화 시키는 것이다.
- 비지도학습이란 실제 결과값 없이 data만으로 알고리즘이 입력값의 특징을 추출하여 알아서 학습하는 것이다.

# 문제 2
### (1) sklearn 패키지에 내장된 breaset_cancer() 데이터를 이용하여
### 학습데이터와 테스트 데이터(7:3 비율)로 분할하세요.

- 분할에 필요한 모듈들은 직접 import 하세요
- 분할 시 random_state은 본인 학번의 뒷 4자리 이용

In [4]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [10]:
X = load_breast_cancer().data
y = load_breast_cancer().target

In [11]:
X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=0.3)

### (2) 분할된 train set의 크기는 얼마인지 출력하세요 (정답은 398이 나와야 함)

In [12]:
X_train.shape

(398, 30)

## 문제 3
### 문제 2에서 분할한 학습데이터를 기반으로 DecisionTreeClassifier를 학습시키고
### 이 모델의 예측 정확도를 출력하세요.

In [15]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
df_clf = DecisionTreeClassifier(random_state=11)
df_clf.fit(X_train,y_train)
predicted = df_clf.predict(X_test)
print("{0:.2f}".format(accuracy_score(y_test,predicted)))

0.93


## 문제 4
### (1) for 문을 이용하여 breast_cancer 데이터에 대해  Stratified K Fold를 수행하세요. (K = 5)
- for 문을 이용한 Stratified K fold
- 출력 1) 교재 109p를 참고하여 학습, 검증 데이터의 분포를 확인
- 출력 2) 교재 111p를 참고하여 각 교차 검증의 정확도와 각 학습, 검증 데이터 크기, 검증 세트 인덱스를 확인

In [49]:
# 출력 1) 교재 109p를 참고하여 학습, 검증 데이터의 분포를 확인
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
import pandas as pd
kfold = KFold(n_splits=5)
skf = StratifiedKFold(n_splits=5)

In [47]:
cv_accuracy=[]
for train_index, test_index in kfold.split(X):
    print(pd.Series(y[train_index]).value_counts())
    print(pd.Series(y[test_index]).value_counts())

1    311
0    144
dtype: int64
0    68
1    46
dtype: int64
1    292
0    163
dtype: int64
1    65
0    49
dtype: int64
1    283
0    172
dtype: int64
1    74
0    40
dtype: int64
1    272
0    183
dtype: int64
1    85
0    29
dtype: int64
1    270
0    186
dtype: int64
1    87
0    26
dtype: int64


In [52]:
for train_index, test_index in skf.split(X,y):
    print(pd.Series(y[train_index]).value_counts())
    print(pd.Series(y[test_index]).value_counts())

1    286
0    169
dtype: int64
1    71
0    43
dtype: int64
1    286
0    169
dtype: int64
1    71
0    43
dtype: int64
1    285
0    170
dtype: int64
1    72
0    42
dtype: int64
1    285
0    170
dtype: int64
1    72
0    42
dtype: int64
1    286
0    170
dtype: int64
1    71
0    42
dtype: int64


In [59]:
# 출력 2) 교재 111p를 참고하여 각 교차 검증의 정확도와 각 학습, 검증 데이터 크기, 검증 세트 인덱스를 확인
import numpy as np
for train_index, test_index in skf.split(X,y):
    x_train,x_test,y_train,y_test=X[train_index],X[test_index],y[train_index],y[test_index]
    df_clf.fit(x_train,y_train)
    predicted = df_clf.predict(x_test)
    print("정확도 : {:.2f}".format(accuracy_score(predicted,y_test)))
    #print(test_index)

정확도 : 0.91
정확도 : 0.90
정확도 : 0.93
정확도 : 0.94
정확도 : 0.89


### (2) cross_val_score()를 이용하여 breast_cancer 데이터에 대해  Stratified K Fold를 수행하세요. (K = 5)
- 출력 1) 교차 검증별 정확도: [0.XXXX, 0.XXXX, 0.XXXX, 0.XXXX]
- 출력 2) 평균 검증 정확도: 0.XXXX

In [60]:
from sklearn.model_selection import cross_val_score, cross_validate

In [71]:
scores=np.round(cross_val_score(df_clf,X,y,scoring="accuracy",cv=5),4)
print("출력 검증별 정확도 : {}".format(scores))
print(f"평균 검증 정확도 : {np.round(np.mean(scores),4)}")

출력 검증별 정확도 : [0.9123 0.9035 0.9298 0.9386 0.8938]
평균 검증 정확도 : 0.9156


## 문제 5
### (1) GridSearchCV 를 이용해 결정트리 알고리즘의 최적의 파라미터 조합을 찾으세요.
- 학습 데이터와 테스트 데이터를 7:3 으로 나누세요.
- parameter: max_depth은 [1, 2, 3, 4] / min_samples_split은 [2, 3]
- 아래와 같이 최적의 조합을 나타내는 표를 찾으세요.
- Stratified 5 fold로 진행하세요. (즉, K=5)

params|mean_test_score|rank_test_score|split0_test_score|split1_test_score|split2_test_score|
    :-:|:-:|:-:|:-:|:-:|:-:|
    {'max_depth': 1, 'min_samples_split': 2}|0.891960|7|0.894737|0.872180|0.909091|
    ...|...|...|...|...|...|
    ...|...|...|...|...|...|
    ...|...|...|...|...|...|
    {'max_depth': 4, 'min_samples_split': 3}|0.899497|3|0.879699|0.887218|0.931818|

In [73]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

In [74]:
import pandas as pd
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)
par = {"max_depth":[1,2,3,4],"min_samples_split":[2,3]}
grid_tree=GridSearchCV(df_clf,param_grid=par,cv=5,refit=True)
grid_tree.fit(X_train,y_train)
scores=pd.DataFrame(grid_tree.cv_results_)

In [76]:
scores[["params","mean_test_score","rank_test_score","split0_test_score","split1_test_score","split2_test_score"]]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.929684,1,0.95,0.9000,0.9250
1,"{'max_depth': 1, 'min_samples_split': 3}",0.929684,1,0.95,0.9000,0.9250
2,"{'max_depth': 2, 'min_samples_split': 2}",0.929589,3,0.95,0.9375,0.9250
3,"{'max_depth': 2, 'min_samples_split': 3}",0.929589,3,0.95,0.9375,0.9250
4,"{'max_depth': 3, 'min_samples_split': 2}",0.927120,5,0.95,0.9125,0.9250
5,"{'max_depth': 3, 'min_samples_split': 3}",0.927120,5,0.95,0.9125,0.9250
6,"{'max_depth': 4, 'min_samples_split': 2}",0.924462,7,0.95,0.9500,0.9375
7,"{'max_depth': 4, 'min_samples_split': 3}",0.924462,7,0.95,0.9500,0.9375


### (2)GridSearchCV를 통해 학습한 estimator를 이용하여 테스트 데이터 셋에 대해 성능을 평가하세요.
- 아래의 두 가지를 출력하세요.
- 출력 1) 테스트 데이터 세트 정확도: 0.XXXX
- 출력 2) GridSearchCV 최적 파라미터: {'max_depth': ?, 'min_samples_split': ?}  <- 물음표 자리에는 최적 파라미터 값이 들어가면 됩니다.

In [78]:
predict = grid_tree.best_estimator_
predicted = predict.predict(X_test)
print(accuracy_score(predicted,y_test))

0.8830409356725146


In [82]:
print(grid_tree.best_params_)
print(grid_tree.best_score_)

{'max_depth': 1, 'min_samples_split': 2}
0.9296835443037976
